In [ ]:
! pip install tensorflow

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

df_train = pd.read_csv("train_submission.csv", usecols=['Text', 'Label'])

# Encoding labels into numeric values
le = LabelEncoder()
df_train['Label'] = le.fit_transform(df_train['Label'])  
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# Splitting into training and validation sets (80% train, 20% validation)
X_train, X_valid, y_train, y_valid = train_test_split(
    df_train["Text"], df_train["Label"], test_size=0.2, random_state=42
)

# Convert labels to NumPy arrays
y_train = y_train.astype("int32")
y_valid = y_valid.astype("int32")

num_classes = df_train['Label'].nunique()

#  Text vectorization layer
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=MAX_VOCAB_SIZE, output_mode="int", output_sequence_length=MAX_SEQUENCE_LENGTH
)


vectorize_layer.adapt(X_train.values)

# Using Embedding
input_text = keras.layers.Input(shape=(), dtype=tf.string, name="text_input")
vectorized_text = vectorize_layer(input_text)  # Convert text to tokenized sequences
embedding = keras.layers.Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=50)(vectorized_text)  
flatten = keras.layers.GlobalAveragePooling1D()(embedding)  
dense1 = keras.layers.Dense(16, activation='relu')(flatten)  # Hidden layer 1
dense2 = keras.layers.Dense(16, activation='relu')(dense1)  # Hidden layer 2
output = keras.layers.Dense(num_classes, activation='softmax')(dense2)  # Output layer


model = keras.Model(inputs=input_text, outputs=output)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)         │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 50)        │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 50)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 390)            │         6,630 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 507,718 (1.94 MB)

 Trainable params: 507,718 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf

#  TensorFlow dataset that includes both inputs (X_train) and labels (y_train)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

# Batch the datasets for efficient training
BATCH_SIZE = 16
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# training
model.fit(
    train_ds,
    epochs=80,  
    validation_data=valid_ds
)


Epoch 1/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 108s 11ms/step - accuracy: 0.0118 - loss: 5.6669 - val_accuracy: 0.0311 - val_loss: 4.9008
Epoch 2/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.0434 - loss: 4.7466 - val_accuracy: 0.0831 - val_loss: 4.2835
Epoch 3/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 123s 13ms/step - accuracy: 0.1019 - loss: 4.1332 - val_accuracy: 0.1373 - val_loss: 3.8781
Epoch 4/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 121s 13ms/step - accuracy: 0.1703 - loss: 3.6982 - val_accuracy: 0.2038 - val_loss: 3.5270
Epoch 5/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 122s 13ms/step - accuracy: 0.2392 - loss: 3.3267 - val_accuracy: 0.2578 - val_loss: 3.2591
Epoch 6/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 117s 12ms/step - accuracy: 0.2962 - loss: 3.0428 - val_accuracy: 0.2965 - val_loss: 3.0778
Epoch 7/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 113s 12ms/step - accuracy: 0.3438 - loss: 2.8185 - val_accuracy: 0.3434 - val_loss: 2.8634
Epoch 8/80
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 136s 14ms/step - accuracy: 0.

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint


train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

BATCH_SIZE = 16
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#  Define the callback to save weights after each epoch
checkpoint_callback = ModelCheckpoint(
    filepath="checkpoints/epoch_{epoch:02d}.weights.h5",  
    save_weights_only=True, 
    verbose=1 
)


history = model.fit(
    train_ds,
    epochs=30,  
    validation_data=valid_ds,
    callbacks=[checkpoint_callback]  # Save weights after each epoch
)


Epoch 1/30
9527/9530 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7730 - loss: 0.9385
Epoch 1: saving model to checkpoints/epoch_01.weights.h5
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.7730 - loss: 0.9385 - val_accuracy: 0.6418 - val_loss: 1.8297
Epoch 2/30
9522/9530 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7731 - loss: 0.9368
Epoch 2: saving model to checkpoints/epoch_02.weights.h5
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.7731 - loss: 0.9368 - val_accuracy: 0.6419 - val_loss: 1.8599
Epoch 3/30
9528/9530 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7735 - loss: 0.9364
Epoch 3: saving model to checkpoints/epoch_03.weights.h5
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.7735 - loss: 0.9364 - val_accuracy: 0.6411 - val_loss: 1.8761
Epoch 4/30
9524/9530 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7743 - loss: 0.9351
Epoch 4: saving model to checkpoints/epoch_04.weights.h5
9530/9530 ━━━━━━━━━━━━━━━━━━━━ 53s 6ms/step - accuracy: 0.77

KeyboardInterrupt: 

In [5]:
model.load_weights("checkpoints/epoch_01.weights.h5")


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
import pandas as pd

# Load the test dataset 
df_test = pd.read_csv("test_without_labels.csv", usecols=['Text'])

# Extract test text data
X_test = df_test["Text"].values




In [8]:
# Get predictions
y_pred_probs = model.predict(df_test["Text"].values) 

# Convert probabilities to class labels
y_pred = y_pred_probs.argmax(axis=1)

# Convert numerical labels back to original text labels
y_pred_labels = le.inverse_transform(y_pred)

df_test = df_test.reset_index(drop=True)  
df_test["ID"] = df_test.index + 1 

 
test_submission = pd.DataFrame(
    {"ID": df_test["ID"], "Label": y_pred_labels}
)


test_submission.to_csv("test_predictions.csv", index=False)

print(test_submission.head()) 


5956/5956 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step
   ID Label
0   1   ven
1   2   epo
2   3   urd
3   4   que
4   5   chk
